# Experiments Premier Analysis on Azure Machine Learning
### Baseline models

In [1]:
import argparse
import os
import sklearn
import pandas as pd 
import numpy as np
from sklearn.metrics import f1_score,accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from azureml.core import Run, Dataset, Environment,Experiment,ScriptRunConfig
from sklearn.preprocessing import LabelEncoder
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.runconfig import DEFAULT_CPU_IMAGE

In [2]:
from azureml.core import  Workspace
from azureml.core.authentication import InteractiveLoginAuthentication
interactive_auth = InteractiveLoginAuthentication(tenant_id="9ce70869-60db-44fd-abe8-d2767077fc8f")

ws = Workspace.from_config()

In [3]:
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

Workspace name: cdh-azml-dev-mlw
Azure region: eastus
Subscription id: 320d8d57-c87c-4434-827f-59ee7d86687a
Resource group: csels-cdh-dev


In [4]:
# current working directory
path = os.getcwd()
print("Current Directory:", path)
  
# parent directory
parent = os.path.join(path, os.pardir)
  
# prints parent directory
print("\nParent Directory:", os.path.abspath(parent))

premier_path = os.path.abspath(parent)

Current Directory: /mnt/batch/tasks/shared/LS_root/mounts/clusters/wsn8-su2/code/Users/WSN8-SU/premier_analysis/azure_ml

Parent Directory: /mnt/batch/tasks/shared/LS_root/mounts/clusters/wsn8-su2/code/Users/WSN8-SU/premier_analysis


### Create Compute

In [5]:
clustername = 'StandardDS3v2'
is_new_cluster = False
try:
    aml_cluster = ComputeTarget(workspace = ws,name= clustername)
    print("Find the existing cluster")
except ComputeTargetException:
    print("Cluster not find - Creating cluster.....")
    is_new_cluster = True
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_DS3_v2',
                                                           max_nodes=4)
    aml_cluster = ComputeTarget.create(ws, clustername, compute_config)

aml_cluster.wait_for_completion(show_output=True)

Find the existing cluster
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


### Upload data

In [6]:
features_path = os.path.join(premier_path,'output/parquet')
features_pkl = os.path.join(premier_path,'output/pkl')


data_store = ws.get_default_datastore()
data_store.upload(src_dir=features_path,target_path='parquet',overwrite=True,show_progress=True)
data_store.upload(src_dir=features_pkl,target_path='pkl',overwrite=True,show_progress=True)

"Datastore.upload" is deprecated after version 1.0.69. Please use "Dataset.File.upload_directory" to upload your files             from a local directory and create FileDataset in single method call. See Dataset API change notice at https://aka.ms/dataset-deprecation.


Uploading an estimated of 3 files
Uploading /mnt/batch/tasks/shared/LS_root/mounts/clusters/wsn8-su2/code/Users/WSN8-SU/premier_analysis/output/parquet/.amlignore
Uploaded /mnt/batch/tasks/shared/LS_root/mounts/clusters/wsn8-su2/code/Users/WSN8-SU/premier_analysis/output/parquet/.amlignore, 1 files out of an estimated total of 3
Uploading /mnt/batch/tasks/shared/LS_root/mounts/clusters/wsn8-su2/code/Users/WSN8-SU/premier_analysis/output/parquet/.amlignore.amltmp
Uploaded /mnt/batch/tasks/shared/LS_root/mounts/clusters/wsn8-su2/code/Users/WSN8-SU/premier_analysis/output/parquet/.amlignore.amltmp, 2 files out of an estimated total of 3
Uploading /mnt/batch/tasks/shared/LS_root/mounts/clusters/wsn8-su2/code/Users/WSN8-SU/premier_analysis/output/parquet/flat_features.parquet
Uploaded /mnt/batch/tasks/shared/LS_root/mounts/clusters/wsn8-su2/code/Users/WSN8-SU/premier_analysis/output/parquet/flat_features.parquet, 3 files out of an estimated total of 3
Uploaded 3 files
Uploading an estimated

$AZUREML_DATAREFERENCE_b316ff3989b94fbbb31f5761241683f7

In [7]:
cohort_path = os.path.join(premier_path,'output/cohort')
data_store.upload(src_dir=cohort_path,target_path='cohort',overwrite=True,show_progress=True)

UserErrorException: UserErrorException:
	Message: src_path must be a directory.
	InnerException None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "src_path must be a directory."
    }
}

In [85]:
%%writefile conda_dependencies_baseline.yml

channels:
- anaconda
- default
dependencies:
- python=3.8
- pip:
  - azureml-defaults
  - matplotlib
  - pandas
  - argparse
  - joblib
  - scikit-learn
  - azureml-sdk
  - openpyxl

Overwriting conda_dependencies_baseline.yml


In [ ]:
premier_train_baseline_env = Environment.from_conda_specification(name='premier_train_baseline_env', file_path='conda_dependencies_baseline.yml')
# Specify a CPU base image
premier_train_baseline_env.docker.enabled = True
premier_train_baseline_env.docker.base_image = DEFAULT_CPU_IMAGE
premier_train_baseline_env.register(workspace=ws)

In [ ]:
experiment = Experiment(workspace=ws, name="Premier-Baseline-models")

In [107]:
# create script config

#mod_names = ['lgr', 'rf', 'gbc', 'svm']
mod_names = ['svm']
for mod in mod_names:
    estimator = ScriptRunConfig(source_directory='./training',
                          script='train_baseline.py',
                          compute_target=aml_cluster,
                           arguments=['--day_one','--outcome', 'icu','--model', mod],
                          environment=premier_train_baseline_env)
    # Create experiment
    experiment = Experiment(workspace=ws, name=f"Premier-Baseline-model-{mod}")
    
    print("Submit Experiment")
    run = experiment.submit(estimator)


Submit Experiment
Submit Experiment
Submit Experiment
Submit Experiment


In [106]:

#run.wait_for_completion(show_output=False,
#                          wait_post_processing=False,
#                         raise_on_error=True)



Submit Experiment


{'runId': 'Premier-Baseline-models_1663976158_1e055d5e',
 'target': 'StandardDS3v2',
 'status': 'Completed',
 'startTimeUtc': '2022-09-23T23:36:19.750038Z',
 'endTimeUtc': '2022-09-23T23:45:34.036832Z',
 'services': {},
 'properties': {'_azureml.ComputeTargetType': 'amlctrain',
  'ContentSnapshotId': '25b3bf30-91a2-40a3-8c18-295a34b26852',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'script': 'train_baseline.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': ['--day_one', '--outcome', 'misa_pt'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'StandardDS3v2',
  'dataReferences': {},
  'data': {},
  'outputData': {},
  'datacaches': [],
  'jobName': None,
  'maxRunDurationSeconds': 2592000,
  'nodeCount': 1,
  'instanceTypes': [],
  'priority': None,
  'credentialPassthrough': False,
  'id